<a href="https://colab.research.google.com/github/enzocfb/general/blob/main/eda_2023_deductivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
"""
Analisis de Deductivos para analizar causas
Impacto :

input:
      Consolidato total de Evaluados por UGS(Formato 06) para todos los años
        Consolidado que contiene los portafolios : 2024-02 , 2024-01, 2023-01 , 2022-02

      Deductivos reportados por UGT
      Fechas referenciales por grupos

output:
      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"

"""

'\nAnalisis de Deductivos para analizar causas\nImpacto :\n\ninput:\n      Consolidato total de Evaluados por UGS(Formato 06) para todos los años\n        Consolidado que contiene los portafolios : 2024-02 , 2024-01, 2023-01 , 2022-02\n\n      Deductivos reportados por UGT\n      Fechas referenciales por grupos\n\noutput:\n      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"\n\n'

In [19]:
import pandas as pd
import numpy as np


In [20]:
####################################################3
# Input  / Output
####################################################3

# Archivo completo de las evaluaciones realizadas por UGS, a partir de la cual se extrae la informacion del beneficiario,
# # Alcance : REGISTRADO- desde que se inicia en ESTADO REGISTRADO, LUEGO A ESTADO EVALUADO , luego  ESTADO VIABILIDAD y finalmente # a ESTADO DE DEDUCTIVIDAD
# KEY or INDICE del ARCHIVO:   CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO)
#                              CONVENIO,  es una agrupacion de Beneficiarios (FALTA, no tiene en este momento esa data)

file = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Formato06Completo.xlsx'

# 2. ARCHIVO DE DEDUCTIVOS (BENEFICIARIO QUE ESTA EN ESTADO = "DEDUCTIVO", es aquel que "SE EXCLUYE" DEL PROGRAMA. Este archivo es recibido de  UGT que contiene la informacion de todos aquellos que fueron
# ALCANCE : VIABLE-->DEDUCTIVO
# KEY or INDICE del ARCHIVO:
#                              CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO) (SI TIENE)
#                              CONVENIO,  es una agrupacion de Beneficiarios,                                               (SI TIENE)

file_deductivos= '/content/drive/MyDrive/@@@Dev/mvcs/Data/LISTADO DE DEDUCTIVOS A (UGS).xlsx'


# Informacion a nivel de nucleos de las fechas de exp tecnico y fecha de Acta de inicio de Construccion
# Alcance : REGISTRADO-->EVALUADO-->VIABLE (TODOs: ESTADO_VIABLE = "VIABLE", INCLUIDO ESTADO_DEDUCTIVO = {"DEDUCTIVO", "NO DEDUCTIVO"] se une a nivel con archivo 2, con los
# 17 nov : Solo se tienen info para VIABLE-->DEDUCTIVO, falta a nivel de todos los VIABLES.
# KEY or INDICE del ARCHIVO:   CONVENIO,  es una agrupacion de Beneficiarios, (SI TIENE)

file_input_nucleo = '/content/drive/MyDrive/@@@Dev/mvcs/Data/NUCLEO - REPORTE COMPLEMENTARIO.xlsx'
file_input_2020_02 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Pnvr_formato06_20231122-15.57.07.xlsx'

# output temporales
file_output_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs no encontrados.xlsx'
file_output_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs si encontrados.xlsx'
file_output_3 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp.xlsx'

file_temp_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_1.xlsx'
file_temp_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_2.xlsx'

# output
file_output_analisis = '/content/drive/MyDrive/@@@Dev/mvcs/Data/deductivos-completo-analysis.xlsx'


# Constantes
constante_origen_PP068 = 'PP068'   # NUMERO DEL PROGRAMA ORIGEN

# Creamos ESTADOS : REGISTRADO, EVALUACION,  VIABILIDAD,
constante_estado_registrado = 'REGISTRADO'

constante_estado_evaluacion_evaluado = "EVALUADO"
constante_estado_evaluacion_rechaza = "RECHAZA_EVALUACION"
constante_estado_evaluacion_otros = 'OTROS_NO_EVALUADO'

constante_estado_viabilidad_viable = "VIABLE"
constante_estado_viabilidad_no_viable = "NO VIABLE"
constante_estado_sininfo = "SIN INFORMACION"
constante_estado_SI = "SI"
constante_estado_SI = "NO"


constante_estado_deductivo = "DEDUCTIVO"
constante_estado_no_deductivo = "NO_DEDUCTIVO"



In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
def convert_to_date(value):
    try:
        return pd.to_datetime(value)
    except ValueError:
        # Handle the case where parsing the date fails
        return value

# # Read the Excel file with the converters parameter
# df = pd.read_excel(file, converters={"Your_Column_Name": convert_to_date})

In [23]:
# La hoja excel, original se le transforma para poder ser leida
# Combinar celdas
# subir fila 6,
# bajamos los nombre de capos de la fila 5 a fila 6 (solo los que faltan)


df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str})
# df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str, "FEC_EVALUACION": convert_to_date})
df.head(3)

# CREACION DE ESTADOS PARA TABLA INICIAL



,NUMERO,ORIGEN,PORTAFOLIO,PORTAFOLIO_T,GRUPO,NRO ENCUESTA,CUV,CODIGO_CCPP,UBIGEO_temp,UBIGEO,...,CRITERIO_SOCIAL,CRITERIO_TECNICO,SOCIAL ESTADO DE LA FICHA,TÉCNICA,OBSERVACIONES,FECHA CADUCIDAD CSE,FECHA_CONSULTA_CSE,ACEPTA_ENTREVISTA,ESTADO_EVALUADO_T,FEC_EVALUACION
0,1,PP068,02-2024,2024-02,2,1,0808050259454185562402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2028-08-19 00:00:00,2023-09-21 02:30:56,SI,SI,14-09-2023
1,2,PP068,02-2024,2024-02,2,2,0808050259000000002402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,SIN REGISTRO,NaN,NaN,NaN,Familia Ausente,NO,14-09-2023
2,3,PP068,02-2024,2024-02,2,3,0808050259475809342402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2024-10-09 00:00:00,2023-09-21 02:37:17,SI,SI,14-09-2023


In [24]:
# Seleccionar campos requeridos para el analisis y construir las features

columns_selected_df = ['NUMERO', 'ORIGEN', 'PORTAFOLIO_T', 'CUV', 'CODIGO_CCPP', 'UBIGEO', 'DEPARTAMENTO','PROVINCIA', 'DISTRITO', 'CENTRO_POBLADO', 'COMUNIDAD', 'PATERNO_JEFE_HOGAR', 'NOMBRES_JEFE_HOGAR', 'DNI_JEFE_HOGAR', 'SEXO_JEFE_HOGAR', 'EDAD_JEFE_HOGAR', 'FEC_NAC_JEFE_HOGAR', 'CANT', 'CTD_DISCAPACIDAD', 'ALTURA', 'LATITUD', 'LONGITUD', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA', 'ACEPTA_ENTREVISTA', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO', 'FEC_EVALUACION', 'FECHA_CONSULTA_CSE']
df = df[columns_selected_df]

# renonmbar dni coyg por DNI_CYG

columns_selected_df_2020_02 = ['CODIGO CUV', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO',
															'CENTRO POBLADO', 'COMUNIDAD', 'PATERNO', 'MATERNO', 'NOMBRES', 'DNI', 'SEXO',
                               'ALTURA','LATITUD', 'LONGITUD', 'DETERMINACIÓN DE LA VIABILIDAD', 'CRITERIO SOCIAL', 'CRITERIO TÉCNICO',
															'VIABILIDAD SOCIAL', 'VIABILIDAD TECNICA',
															'DNI_CYG',
															'CLASIFICACION SOCIOECONOMICA CSE'
															]


In [25]:
#

def cargar_datos_portafolio_2022_2(file_input, df, columns_selected_df_2020_02):

  df_2020_2 = pd.read_excel(file_input, skiprows=5)
  print(df_2020_2.head(3))

  df_2020_2 = df_2020_2[columns_selected_df_2020_02]

  # df_2020_2.to_excel(file_output_3)
  print(df_2020_2.info())

  # Mostrar el DataFrame resultante
  # print(df_2020_2.head(3))

  # Se adicionaran las siguientes columnas:

  # NUMERO
  # ORIGEN
  # PORTAFOLIO_T
  # CODIGO_CCPP
  # UBIGEO
  # EDAD_JEFE_HOGAR
  # FEC_NAC_JEFE_HOGAR
  # CANT
  # CTD_DISCAPACIDAD
  # FEC_EVALUACION
  # FECHA_CONSULTA_CSE

  new_columns_df_2020_2 = [
  'CODIGO_CCPP', 'UBIGEO', 'EDAD_JEFE_HOGAR', 'FEC_NAC_JEFE_HOGAR','CANT','CTD_DISCAPACIDAD','FEC_EVALUACION','FECHA_CONSULTA_CSE'
  ]

  # Add empty columns to the DataFrame
  for col_name in new_columns_df_2020_2:
      df_2020_2[col_name] = None

  # renombar las columnas campos segun el df padre : df:

  df_2020_2.rename(columns={'CODIGO CUV': 'CUV'}, inplace=True)
  df_2020_2.rename(columns={'CENTRO POBLADO': 'CENTRO_POBLADO'}, inplace=True)
  df_2020_2.rename(columns={'PATERNO': 'PATERNO_JEFE_HOGAR'}, inplace=True)
  df_2020_2.rename(columns={'NOMBRES': 'NOMBRES_JEFE_HOGAR'}, inplace=True)
  df_2020_2.rename(columns={'DNI': 'DNI_JEFE_HOGAR'}, inplace=True)
  df_2020_2.rename(columns={'SEXO': 'SEXO_JEFE_HOGAR'}, inplace=True)
  df_2020_2.rename(columns={'CLASIFICACION SOCIOECONOMICA CSE': 'CLASIFICACION_SOCIOECONOMICA_CSE'}, inplace=True)


  df_2020_2.rename(columns={'DETERMINACIÓN DE LA VIABILIDAD': 'VIABILIDAD_TOTAL'}, inplace=True)
  df_2020_2.rename(columns={'VIABILIDAD SOCIAL': 'VIABILIDAD_SOCIAL'}, inplace=True)
  df_2020_2.rename(columns={'VIABILIDAD TECNICA': 'VIABILIDAD_TECNICA'}, inplace=True)
  df_2020_2.rename(columns={'CRITERIO SOCIAL': 'CRITERIO_SOCIAL'}, inplace=True)
  df_2020_2.rename(columns={'CRITERIO TÉCNICO': 'CRITERIO_TECNICO'}, inplace=True)

  condition_1 = ( df_2020_2['VIABILIDAD_TOTAL'] == "VIABLE") | (df_2020_2['VIABILIDAD_TOTAL'] == "NO VIABLE")
  # condition = (df['A'] == 2) | (df['B'] > 30)

  df_2020_2['ACEPTA_ENTREVISTA']=  np.where(condition_1, constante_estado_SI, constante_estado_sininfo )

  df_2020_2['NUMERO']= range(1, len(df_2020_2) + 1)
  df_2020_2['ORIGEN']= constante_origen_PP068

  # Crear una nueva columna llamada 'Columna3' que sea la concatenación de "20" y las últimas dos posiciones de 'Columna1'
  df_2020_2['PORTAFOLIO_T'] = '20' + df_2020_2['CUV'].str[-4:-2] + '-' +  df_2020_2['CUV'].str[-2:]

  df_2020_2 = df_2020_2[columns_selected_df]



  df_temp_df = df.sample(n=10)
  df_temp_df2 = df_2020_2.sample(n=10)
  df_temp_df.to_excel(file_temp_1)
  df_temp_df2.to_excel(file_temp_2)


  df_total = pd.concat([df, df_2020_2])

  print('************** Resultados de Dataset base final ****')
  print(df.shape[0])
  print(df_2020_2.shape[0])
  print(df_total.shape[0])

  return df_total


In [26]:
# mask = df_2020_2.index != 0  # Index 2 corresponds to the third row (indexing starts from 0)

# Use boolean indexing to select rows that satisfy the mask
# df_2020_2 = df_2020_2[mask]


In [27]:

def add_columns_to_df(df):

  df['ESTADIO_REGISTRADO'] = constante_estado_registrado
  df['ESTADO_EVALUACION'] = df['ACEPTA_ENTREVISTA'].apply(lambda x: 'SI' if x == "SI" else 'NO')

  df['ESTADO_EVALUACION'] = np.where(df['ACEPTA_ENTREVISTA'] == 'SI', constante_estado_evaluacion_evaluado,
                          np.where(df['ACEPTA_ENTREVISTA'] == 'Rechazo de evaluación / Intervención', constante_estado_evaluacion_rechaza, constante_estado_evaluacion_otros))


  # df['ESTADO_VIABLE'].fillna(constante_estado_viabilidad_sininfo, inplace = True)

  df['ESTADO_VIABLE'] = np.where(df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable, constante_estado_viabilidad_viable,
                          np.where(df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_no_viable, constante_estado_viabilidad_no_viable, constante_estado_sininfo))

  # print(df.info())
  # print(df.head(3))

  return df

In [28]:
df = cargar_datos_portafolio_2022_2(file_input_2020_02, df, columns_selected_df_2020_02)





   GRUPO  NRO ENCUESTA              CODIGO CUV DEPARTAMENTO    PROVINCIA  \
0     20            26  0302010020449973362002     APURIMAC  ANDAHUAYLAS   
1     20            27  0302010020421954282002     APURIMAC  ANDAHUAYLAS   
2     20            28  0302010020409033352002     APURIMAC  ANDAHUAYLAS   

      DISTRITO CENTRO POBLADO     COMUNIDAD  PATERNO MATERNO  ...  \
0  ANDAHUAYLAS       MOROCOSE  SOCCÑACANCHA  ANDRADA  ALLCCA  ...   
1  ANDAHUAYLAS       MOROCOSE  SOCCÑACANCHA   HUASCO   POCCO  ...   
2  ANDAHUAYLAS       MOROCOSE  SOCCÑACANCHA   CUEVAS  ALLCCA  ...   

        Unnamed: 28  CLASIFICACION SOCIOECONOMICA CSE  \
0  TERRENO VIVIENDA                     POBRE EXTREMO   
1  TERRENO VIVIENDA                     POBRE EXTREMO   
2  TERRENO VIVIENDA                     POBRE EXTREMO   

  DETERMINACIÓN DE LA VIABILIDAD   VIABILIDAD SOCIAL VIABILIDAD TECNICA  \
0                         VIABLE  SOCIALMENTE VIABLE             VIABLE   
1                         VIABLE  SOCIA

In [29]:
df = add_columns_to_df(df)

In [ ]:
##############################################################
# DEDUCTIVOS
##############################################################



def crear_dataset_deductivos(file_input_deductivos, file_input_nucleo):

  df_deductivos = pd.read_excel(file_input_deductivos)
  df_nucleo = pd.read_excel(file_input_nucleo)

  print(df_nucleo.head(3))

  df_deductivos_nucleo = pd.merge(df_deductivos, df_nucleo,  on = 'CONVENIO', how = 'left')

  # ESTADO : CREAMOS EL ESTADO DEDUCTIVO PARA ESTA TABLA
  df_deductivos_nucleo['ESTADO_DEDUCTIVO'] = 'DEDUCTIVO'

  print(df_deductivos.shape[0])
  print(df_deductivos_nucleo.shape[0])
  # print(df_deductivos_nucleo.head())

  df_deductivos_nucleo.to_excel(file_output_3)
  # print(df_deductivos_nucleo.info())

  columns_selected_deductivos = ['ID', 'CUV','DNI','CODIGO_UGT','CONVENIO','NOMBRE','AP_PATERNO','DEDUCTIVO','CARTA','MOTIVO','HT','FEC_APROB_EXP_TECNICO','FEC_ACTA_INICIO_CONSTRUCCION','ESTADO_DEDUCTIVO']

  df_deductivos_nucleo = df_deductivos_nucleo[columns_selected_deductivos]
  # 130 registros

  return df_deductivos_nucleo


def union_df_with_deductivos(df, df_deductivos_nucleo):

  df_join = pd.merge(df, df_deductivos_nucleo, on = 'CUV')
  df_join_left = pd.merge(df, df_deductivos_nucleo, on = 'CUV', how = 'left', suffixes=('_F06', '_deductivo'))

  print(df_join_left['FEC_EVALUACION'])

  return df_join_left

In [ ]:
df_deductivos = crear_dataset_deductivos(file_deductivos, file_input_nucleo)
df_join_left = union_df_with_deductivos(df,df_deductivos_nucleo)

In [ ]:
df_join_left = add_and_transform_columns_of_deductivos(df_join_left)

In [ ]:
##########################################################################3
# CALCULOS POST PROCESO
##########################################################################3

# Completar ESTADO DEDUCTIVO , luego del JOIN

def add_and_transform_columns_of_deductivos(df):

  df_temp = df

  condition_1 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & (df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )
  condition_2 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & ~(df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )

  df_temp.loc[condition_1, 'ESTADO_DEDUCTIVO'] = constante_estado_no_deductivo
  df_temp.loc[condition_2, 'ESTADO_DEDUCTIVO'] = constante_estado_sininfo

  # FECHAS:
  # FEC_EVALUACION, FEC_APROB_EXP_TECNICO,FEC_ACTA_INICIO_CONSTRUCCION
  #

    # Convert date strings to datetime objects
  df_temp['FEC_EVALUACION'] = pd.to_datetime(df_temp['FEC_EVALUACION'])
  df_temp['FEC_APROB_EXP_TECNICO'] = pd.to_datetime(df_temp['FEC_APROB_EXP_TECNICO'])
  df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] = pd.to_datetime(df_temp['FEC_ACTA_INICIO_CONSTRUCCION'])

  # Calculate the difference between the two dates
  df_temp['days_diff_exp_eval'] = (df_temp['FEC_APROB_EXP_TECNICO'] - df_temp['FEC_EVALUACION']).dt.days
  df_temp['days_diff_acta_eval'] = (df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] - df_temp['FEC_EVALUACION']).dt.days
  df_temp['days_diff_acta_exp'] = (df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] - df_temp['FEC_APROB_EXP_TECNICO']).dt.days

  print(df_temp.info())

  print(df_temp['days_diff_acta_exp'])

  ############################
  # fillna para las fechas : se completa con 0, las que son nulas.
  # 0, es un valor que en la realidad NO es valido, porque desde la EVALUACION hasta la REALIZACION del expediente, o
  # DESDE la APROBACION DEL EXPEDIENTE TECNICO hasta la CONSTRUCCION, el tiempo es mayor que 0
  ############################

  # List of columns to fill with zeros
  columns_to_fill = ['days_diff_exp_eval', 'days_diff_acta_eval', 'days_diff_acta_exp', 'CANT']

  # Fill NaN values with 0 only in the specified columns
  df_temp[columns_to_fill] = df_temp[columns_to_fill].fillna(0)

  return df_temp

In [ ]:

df_join_left.to_excel(file_output_analisis)

In [ ]:
print(df.shape[0])
print(df_join.shape[0])
print(df_join_left.shape[0])
# df_join.head()
df_join_left.to_excel(file_output_analisis)

28078
130
28078


In [ ]:
df1 = df_deductivos
df2 = df




In [ ]:
rows_not_in_df06 = df1[~df1['CUV'].isin(df2['CUV'])]
rows_in_df06 = df1[df1['CUV'].isin(df2['CUV'])]

rows_not_in_df06.to_excel(file_output_1)
rows_in_df06.to_excel(file_output_2)

In [ ]:
print(rows_not_in_df.shape[0])
print(rows_in_df06.shape[0])

887
130


In [ ]:
import pandas as pd

# Sample DataFrames (Replace these with your actual DataFrames)
data1 = {'key': [1, 2, 3, 4, 5],
         'value': ['A', 'B', 'C', 'D', 'E']}
data2 = {'key': [2, 4, 5],
         'value': ['B', 'D', 'E']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
print(df1)
print(df2)

# Select rows in df1 that are not in df2 based on 'key'
rows_not_in_df2 = df1[~df1['key'].isin(df2['key'])]

print(rows_not_in_df2)
